# Introduction
Microsoft has identified a new business oportunity in the film industry following the success of other major companies producing original films. We will explore high performing film genres and translate them into actionable insights.

## Objectives
- **Analyze Current Box Office trends:** Examine the type of films that are currently succesful at the box office.
- **Identify Key Film Attributes:** Determine the attributes (e.g., genre, target audience, budget range) of the top-performing films
- **Provide Actionable Insights:** Offer recommendations on the type of films Microsoft should produce in the analysis.

## Key Questions
1. **What genres are currently performing the best at the box office?**
2. **What re the common characteristics of the top-grossing films (e.g., budget, cast, director, special effects)?**
3. **Who are the target audiences for these successful films?**
4. **How do seasonal trends affect box office performance?**
5. **What marketing strategies are being used by top-performing films?**

### 1. Data Undestanding
In this part, we will preprocess dataset from Box Office Mojo (Data\bom.movie_gross.csv) which contains data on movies doing well at the box office.

In [1]:
import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

### 1.1) Read `bom.movie_gross.csv` into a pandas DataFrame named `df`

We will use pandas to create a new DataFrame, called `df`, containing the data from the dataset in the file `bom.movie_gross.csv` in the folder containing this notebook. 

In [2]:
df = pd.read_csv('Data/bom.movie_gross.csv')
df.head() # Returns the first five rows from the datafame.

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


In [3]:
df.shape # Returns a tuple representing the rows and columns of the  dataframe.

(3387, 5)

In [4]:
#To check a  summary of the df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


#### From the above code block, we can observe that there are missing values as the number of Non-Null Counts are different. We will handle this in part 2 on Data Preparation.

## Part 1.2  Read the Sql data
Here we will read the sql data from ("C:\Users\ADMIN\Desktop\Data_project1\im.db.db") and try to understand how it relates to the (C:\Users\ADMIN\Desktop\Project\Data\bom.movie_gross.csv).

In [14]:
conn = sqlite3.connect('Data_project1\im.db')